In [ ]:
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)
from keras import losses, models
from keras.activations import relu, softmax
from keras.layers import (Convolution2D, GlobalAveragePooling2D, BatchNormalization, Flatten, Dropout,
                          GlobalMaxPool2D, MaxPool2D, concatenate, Activation, Input, Dense)

from tensorflow.keras.optimizers import Adam 

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm, tqdm_pandas
import scipy
from scipy.stats import skew
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob 
import os
import pickle
import sys
import IPython.display as ipd 
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
import csv
import torch

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib
tf.config.list_physical_devices()

In [ ]:
train = pd.read_csv(r'C:/Data/Sentiment Analysis/MELD/Processed/Processed_final/v4/train.csv')
dev =  pd.read_csv(r'C:/Data/Sentiment Analysis/MELD/Processed/Processed_final/v4/dev.csv')
test =  pd.read_csv(r'C:/Data/Sentiment Analysis/MELD/Processed/Processed_final/v4/test.csv')

In [ ]:
print(len(train))
print(len(dev))
print(len(test))

In [ ]:
dfs = [train,test]
MELD_df = pd.concat(dfs, ignore_index = True)

In [ ]:
print(len(MELD_df))

In [ ]:
MELD_df.head()

In [ ]:

'''
1. Extracting the MFCC feature as an image (Matrix format).  
'''
def prepare_data(df, n, mfcc):
    X = np.empty(shape=(MELD_df.shape[0], n, 1282, 1))
    input_length = sampling_rate * audio_duration
    
    cnt = 0
    for fname in tqdm(MELD_df.path):
        file_path = fname
        data, _ = librosa.load(file_path, sr=sampling_rate
                               ,res_type="kaiser_fast"
                               ,duration=41
                               ,offset=0.0
                              )

        # Random offset / Padding
        if len(data) > input_length:
            max_offset = len(data) - input_length
            offset = np.random.randint(max_offset)
            data = data[offset:(input_length+offset)]
        else:
            if input_length > len(data):
                max_offset = input_length - len(data)
                offset = np.random.randint(max_offset)
            else:
                offset = 0
            data = np.pad(data, (offset, int(input_length) - len(data) - offset), "constant")
        
        # which feature?
        if mfcc == 1:
            # MFCC extraction 
            MFCC = librosa.feature.mfcc(data, sr=sampling_rate, n_mfcc=n_mfcc)
            MFCC = np.expand_dims(MFCC, axis=-1)
            X[cnt,] = MFCC
            
        else:
            # Log-melspectogram
            melspec = librosa.feature.melspectrogram(data, n_mels = n_melspec)   
            logspec = librosa.amplitude_to_db(melspec)
            logspec = np.expand_dims(logspec, axis=-1)
            X[cnt,] = logspec
            
        cnt += 1
    
    return X


'''
2. Confusion matrix plot 
'''        
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    '''Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.

    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.

    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    '''
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")

    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    
    
'''
# 3. Create the 2D CNN model 
'''
def get_2d_conv_model(n):
    ''' Create a standard deep 2D convolutional neural network'''
    nclass = 3
    inp = Input(shape=(n,1282, 1))  #2D matrix of 30 MFCC bands by xxx audio length.
    x = Convolution2D(32, (4,10), padding="same")(inp)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D()(x)
    x = Dropout(rate=0.2)(x)
    
    x = Convolution2D(32, (4,10), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D()(x)
    x = Dropout(rate=0.2)(x)
    
    x = Convolution2D(32, (4,10), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D()(x)
    x = Dropout(rate=0.2)(x)
    
    x = Convolution2D(32, (4,10), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D()(x)
    x = Dropout(rate=0.2)(x)
    
    x = Flatten()(x)
    x = Dense(64)(x)
    x = Dropout(rate=0.2)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(rate=0.2)(x)
    
    out = Dense(nclass, activation=softmax)(x)
    
    model = models.Model(inputs=inp, outputs=out)
    opt = Adam(0.001) #optimizer= "adam"
    model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['acc']) 
    return model

'''
# 4. Other functions 
'''
class get_results:
    '''
    We're going to create a class (blueprint template) for generating the results based on the various model approaches. 
    So instead of repeating the functions each time, we assign the results into on object with its associated variables 
    depending on each combination:
        1) MFCC with no augmentation  
        2) MFCC with augmentation 
        3) Logmelspec with no augmentation 
        4) Logmelspec with augmentation
    '''
    
    def __init__(self, model_history, model ,X_test, y_test, labels):
        self.model_history = model_history
        self.model = model
        self.X_test = X_test
        self.y_test = y_test             
        self.labels = labels

    def create_plot(self, model_history):
        '''Check the logloss of both train and validation, make sure they are close and have plateau'''
        plt.plot(model_history.history['loss'])
        plt.plot(model_history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

    def create_results(self, model):
        '''predict on test set and get accuracy results'''
        opt = Adam(0.001) #optimizer= "adam"
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        score = model.evaluate(X_test, y_test, verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

    def confusion_results(self, X_test, y_test, labels, model):
        '''plot confusion matrix results'''
        preds = model.predict(X_test, 
                                 batch_size=16, 
                                 verbose=2)
        preds= preds.argmax(axis=1)
        preds = preds.astype(int).flatten()
        preds = (lb.inverse_transform((preds)))

        actual = y_test.argmax(axis=1)
        actual = actual.astype(int).flatten()
        actual = (lb.inverse_transform((actual)))

        classes = labels
        classes.sort()    

        c = confusion_matrix(actual, preds)
        print_confusion_matrix(c, class_names = classes)

## MFCC

In [ ]:
sampling_rate=16000
audio_duration=41
n_mfcc = 30
mfcc = prepare_data(MELD_df.path, n = n_mfcc, mfcc = 1)

In [ ]:
# Split between train and test 
X_train, X_test, y_train, y_test = train_test_split(mfcc
                                                    , MELD_df.sentiment
                                                    , train_size=9988
                                                    , test_size=2608
                                                    , shuffle=False
                                                   )


# one hot encode the target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

# Normalization as per the standard NN process
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

# Build CNN model
with tf.device("/gpu:0"):
    model = get_2d_conv_model(n=n_mfcc)
    model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                              batch_size=16, verbose = 2, epochs=20)

In [ ]:
print(X_test.shape, y_test.shape)

In [ ]:
results = get_results(model_history,model,X_test,y_test, MELD_df.sentiment.unique())
results.create_plot(model_history)
results.create_results(model)
results.confusion_results(X_test, y_test, MELD_df.sentiment.unique(), model)

### Storing predictions  

### For test: 

In [ ]:
lb_name_mapping = dict(zip(lb.classes_, lb.transform(lb.classes_)))
print(lb_name_mapping)

In [ ]:
preds_test = model.predict(X_test)
preds_test1 = preds_test.argmax(axis=1).tolist()
actual_test = y_test.argmax(axis=1).tolist()        
fileID = test.name

In [ ]:
test_predictions = { "fileID_mfcc": [], "Negative_mfcc" : [], "Neutral_mfcc" : [], "Positive_mfcc" : [], "predicted_mfcc": [], "actual_mfcc": []}

for i in preds_test:

    test_predictions["Negative_mfcc"].append(i[0])
    test_predictions["Neutral_mfcc"].append(i[1])
    test_predictions["Positive_mfcc"].append(i[2])
    
for i in preds_test1:
    
    test_predictions["predicted_mfcc"].append(i)
    
for i in actual_test:
    
    test_predictions["actual_mfcc"].append(i)
    
for i in test["name"]:

    test_predictions["fileID_mfcc"].append(i)

In [ ]:
df = pd.DataFrame(data=test_predictions)
df

In [ ]:
#saving to csv
df.to_csv(r"C:\Data\Sentiment Analysis\MELD\ensemble preds\2d_cnn\v3\test_mfcc_preds.csv")

## Mel spectogram

In [ ]:
sampling_rate=16000
audio_duration=41
n_melspec = 60
specgram = prepare_data(MELD_df, n = n_melspec, mfcc = 0)

In [ ]:
# Split between train and test 
X_train, X_test, y_train, y_test = train_test_split(specgram
                                                    , MELD_df.sentiment
                                                    , train_size=9988
                                                    , test_size=2608
                                                    , shuffle=False
                                                   )

# one hot encode the target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

# Normalization as per the standard NN process
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

# Build CNN model 
with tf.device("/gpu:0"):
    model = get_2d_conv_model(n=n_melspec)
    model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                              batch_size=16, verbose = 2, epochs=20)

In [ ]:
results = get_results(model_history,model,X_test,y_test, MELD_df.sentiment.unique())
results.create_plot(model_history)
results.create_results(model)
results.confusion_results(X_test, y_test, MELD_df.sentiment.unique(), model)

In [ ]:
print(X_test.shape, y_test.shape)

### Storing predictions 

### For test: 

In [ ]:
lb_name_mapping = dict(zip(lb.classes_, lb.transform(lb.classes_)))
print(lb_name_mapping)

In [ ]:
preds_test = model.predict(X_test)
preds_test1 = preds_test.argmax(axis=1).tolist()
#preds_test = (lb.inverse_transform((preds_test)))

actual_test = y_test.argmax(axis=1).tolist()
#actual_test = (lb.inverse_transform((actual_test)))
        
fileID = test.name

In [ ]:
test_predictions = { "fileID_specgram": [], "Negative_specgram" : [], "Neutral_specgram" : [], "Positive_specgram" : [], "predicted_specgram": [], "actual_specgram": []}

for i in preds_test:

    test_predictions["Negative_specgram"].append(i[0])
    test_predictions["Neutral_specgram"].append(i[1])
    test_predictions["Positive_specgram"].append(i[2])
    
for i in preds_test1:
    
    test_predictions["predicted_specgram"].append(i)
    
for i in actual_test:
    
    test_predictions["actual_specgram"].append(i)
    
for i in test["name"]:

    test_predictions["fileID_specgram"].append(i)

In [ ]:
df = pd.DataFrame(data=test_predictions)
df

In [ ]:
#saving to csv
df.to_csv(r"C:\Data\Sentiment Analysis\MELD\ensemble preds\2d_cnn\v3\test_specgram_preds.csv")